In [ ]:
# default_exp noise.fault_types

# Fault types

> Collection of classes defining types of errors that can happen on circuit elements.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
from qsam.datatypes.circuit import Circuit, partition, unpack
import numpy as np
import itertools as it

In [ ]:
#export
class OneQubitGateFaults:
    
    def generate(self):
        pass

In [ ]:
#export
class TwoQubitGateFaults:
    pass

In [ ]:
#export
class IdleQubitFaults:
    pass

In [ ]:
#export
class OneQubitGateOverrotationFaults:
    pass

In [ ]:
#export
class TwoQubitGateOverrotationFaults:
    pass

In [ ]:
#export
class OneQubitGateCrosstalkFaults:
    pass

In [ ]:
#export
class TwoQubitGateCrosstalkFaults:
    pass

In [ ]:
from qsam.datatypes.circuit import Circuit, unpack

circuit = Circuit([{'init':[0,1,2,3,4,5]}, {'H': [1, 2, 3]}, {'X': [0, 1], 'CNOT': [(2, 3), (1,5)]}, {'measure' : [1,2,3]}])
circuit

0: {'init': [0, 1, 2, 3, 4, 5]}
1: {'H': [1, 2, 3]}
2: {'X': [0, 1], 'CNOT': [(2, 3), (1, 5)]}
3: {'measure': [1, 2, 3]}

In [ ]:
GATESETS = {
    'gate1': {'H', 'X', 'Y', 'Z', 'I', 'S', 'Sd', 'T', 'Td', 'Q', 'Qd', 'R', 'Rd'},
    'gate2': {'CNOT', 'MSd'},
    'meas': {'measure'},
    'init': {'init'}
}
    
def partition(circuit):

    qubits = set(range(circuit.n_qubits))
    out = {'idle': []}
    get_partition = lambda gset: [(i,q) for i,t in enumerate(circuit)
                                   for g,qs in t.items() for q in qs if g in gset]
    
    for gname,gset in GATESETS.items():
        out[gname] = get_partition(gset)
        
    for i,tick in enumerate(circuit):
        out['idle'] += [(i,q) for q in qubits.difference(set(unpack(tick)))]
    
    return out
            
partition(circuit)

{'idle': [(1, 0), (1, 4), (1, 5), (2, 4), (3, 0), (3, 4), (3, 5)],
 'gate1': [(1, 1), (1, 2), (1, 3), (2, 0), (2, 1)],
 'gate2': [(2, (2, 3)), (2, (1, 5))],
 'meas': [(3, 1), (3, 2), (3, 3)],
 'init': [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]}

In [ ]:
def gates_at_loc(circuit, tick_idx, qbloc):
    gates = circuit[tick_idx]
    return [gate for gate, locs in gates.items() if qbloc in locs]


gates_at_loc(circuit, 2, (2,3))
# circuit

['CNOT']

In [ ]:
circuit

0: {'init': [0, 1, 2, 3, 4, 5]}
1: {'H': [1, 2, 3]}
2: {'X': [0, 1], 'CNOT': [(2, 3), (1, 5)]}
3: {'measure': [1, 2, 3]}

In [ ]:
par = partition(circuit)

n_faults = {'gate1': 1, 'gate2': 0, 'idle':2, 'meas': 0, 'init': 0}

# clarify if xtalk1, xtalk2, overrotation should have their own subsets.

def faults_from_weights(weights):
    faults = []
    for group,locs in par.items():
        r_ids = np.random.choice(len(locs), weights[group], replace=False)
        faults += [locs[r_idx] for r_idx in r_ids]
        
    # include overrot and crosstalk..
    return faults

def xt_neighbors(circuit):
    
    out = {'gate1': [], 'gate2': {'common': [], 'uncommon': []}}
    n_qubits = set(range(circuit.n_qubits))
    
    for i,tick in enumerate(circuit):
        
        tickqubits = set(unpack(tick))
        inactive = n_qubits.difference(tickqubits)
        
        for g,qs in tick.items():
            
            if g in GATESETS['gate1']:
                for q in qs:
                    if q-1 in inactive:
                        out['gate1'].append((i,q-1))
                    elif q+1 in inactive:
                        out['gate1'].append((i,q+1))
                        
            elif g in GATESETS['gate2']:
                neighbors = []
                for qtup in qs:
                    neighbor = []
                    for q in qtup:
                        if q-1 in inactive:
                            neighbor.append((i,q-1))
                        elif q+1 in inactive:
                            neighbor.append((i,q+1))
                    neighbors.append(neighbor)
                    
                out['gate2']['common'] = list(filter(lambda x: x in neighbors[1], neighbors[0]))
                out['gate2']['uncommon'] = list(filter(lambda x: x not in neighbors[1], neighbors[0]))
    return out
        
xt_neighbors(circuit)
# faults_from_weights(n_faults)

{'gate1': [(1, 0), (1, 4)], 'gate2': {'common': [(2, 4)], 'uncommon': []}}

In [ ]:
circuit

0: {'init': [0, 1, 2, 3, 4, 5]}
1: {'H': [1, 2, 3]}
2: {'X': [0, 1], 'CNOT': [(2, 3), (1, 5)]}
3: {'measure': [1, 2, 3]}

In [ ]:
l = ['a','b','c']
b = ['c','a']
list(filter(lambda x: x in b, l))

['a', 'c']